## Creating the data for the chatbot
1. Defining intents in **nlu.md**
2. NLU configuration and Policy definition in **config.yml** (includes policies now...earlier policies were defined in policy.yml)
4. Defining stories in **stories.md**
3. Defining domain in **domain.yml**
5. Defining custom action as python code in **actions.py**
6. Visualize the graph of all possible intents and responses -- using domain.yml and stories.md

### 1. Defining intents in nlu.md

In [ ]:
!pip install rasa[full]

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
nlu_md = """

## intent:greet
- hey
- hello
- hi
- Hi

## intent:fine_ask
- I am good, how are you doing?
- I'm fine, how are you?
- I'm good, how are you?

## intent:my_name_is
- I am [Atul](PERSON)
- I am [Sampriti](PERSON)
- I'm [Prerna](PERSON)
- im [Varun](PERSON)
- My name is [Nikhil](PERSON)

## intent:fine_normal
- I am doing great
- I'm doing great
- I'm fine
- I'm good

## intent:news
- Share some latest news around the [world](category)?
- Share some latest news in [sports](category)?
- What is going on in [technology](category)?
- Tell me some news about [fashion](category)
- Tell me some news about [business](category)
- Tell me some news about [arts](category)
- What is going on in [arts](category)
- What is cooking in [food](category)
- [movies](category)

## intent:thanks
- Thanks
- Thank you so much

## intent:bye
- No, I am good as of now. Bye
- Bye
- Bbye
"""


f = open('./data/nlu.md', 'w')
f.write(nlu_md)
f.close()

# %store nlu_md > nlu.md

### 2. NLU configuration and Policy definition

In [11]:
config = """
version: "2.0"
language: "en"

pipeline:
- name: "WhitespaceTokenizer"
- name: "RegexFeaturizer"
- name: "CRFEntityExtractor"
- name: "EntitySynonymMapper"
- name: "CountVectorsFeaturizer"
- name: "CountVectorsFeaturizer"
  analyzer: "char_wb"
  min_ngram: 1
  max_ngram: 4
- name: "EmbeddingIntentClassifier"

policies:
  - name: TEDPolicy
    epochs: 100
    max_history: 3
  - name: MemoizationPolicy
    max_history: 3
  - name: RulePolicy
    nlu_threshold: 0.1
    core_threshold: 0.2
    fallback_action_name: 'utter_default'
  - name: FormPolicy

""" 

f = open('config.yml', 'w')
f.write(config)
f.close()
# %store config > config.yml

### 3. Defining stories

In [12]:
stories_md = """
## fallback
- utter_default

## greeting path 1
* greet
- utter_greet

## my name is
* my_name_is
- utter_its_nice_to_meet_you

## fine path 1
* fine_normal
- utter_help

## fine path 2
* fine_ask
- utter_reply

## news path
* news
- action_get_news
- utter_ofc

## thanks path 1
* thanks
- utter_anything_else

## bye path 1
* bye
- utter_bye
"""

f = open('./data/stories.md', 'w')
f.write(stories_md)
f.close()

#%store stories_md > stories.md

### 4. Defining a domain

In [ ]:
#I am Nachiketh

In [13]:
domain_yml = """

version: "2.0"

slots:
  category:
    type: text
  PERSON:
    type: text

entities:
- category
- PERSON

intents:
- greet
- my_name_is
- fine_ask
- fine_normal
- news
- thanks
- bye

actions:
- action_restart
- action_get_news
- utter_greet
- utter_reply
- utter_help
- utter_anything_else
- utter_ofc
- utter_bye
- utter_default
- utter_its_nice_to_meet_you

templates:

  utter_greet:
    - text: Hey {PERSON}, how are you?
    - text: Hello {PERSON}, How are you doing?
    
  utter_its_nice_to_meet_you:
    - It's nice to meet you, {PERSON}.
    - Nice to meet you, {PERSON}.
  
  utter_reply:
    - text: I'm doing great. Please let me know what I can do for you.
    - text: I'm doing great. Tell me How can I help you today?
  utter_help:
    - text: Great{PERSON}. How can I help you?
    - text: Great. Tell me How can I help you?
    - text: Great. Tell me what all news you would like to get.
  utter_anything_else:
    - text: No worries. Is there anything else I can help you with?
    - text: No worries. Let me know if there is anything else I can help you with
  utter_ofc:
    - text: I can definitely help you. The top 5 news of the {category}
    - text: Surely, I can help you. The top 5 news of the {category}
  utter_bye:
    - text: Bye and have a nice day
    - text: Bbye and have a nice day
  utter_default:
    - text: I am not sure what you're aiming for
    - text: I am sorry but I am not able to get you.
    - text: My appologies but I am not able to get you
"""

f = open('domain.yml', 'w')
f.write(domain_yml)
f.close()
#%store domain_yml > domain.yml

### 5. Defining custom action as python code in actions.py

In [14]:
custom_action = """

from typing import Text, List, Dict, Any
from rasa_sdk import Action, Tracker
from rasa_sdk.events import SlotSet, SessionStarted, ActionExecuted, EventType
from rasa_sdk.executor import CollectingDispatcher
import requests
import json

import requests

class ApiAction(Action):
    def name(self):
        return "action_get_news"

    def run(self,
           dispatcher: CollectingDispatcher,
           tracker: Tracker,
           domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        category = tracker.get_slot('category')
        print(category)
        
        url = 'https://api.nytimes.com/svc/news/v3/content/all/{category}.json'.format(category=category)
        params = {'api-key': "2hq54bvFO0yWiRdY70reBU2GmusBtnwM", 'limit': 5}
        response = requests.get(url, params).text
        json_data = json.loads(response)['results']
        i = 0
        for results in json_data:
            i = i+1
            message = str(i) + "." + results['abstract']
            dispatcher.utter_message(message)
        return[]
        
"""
f = open('actions.py', 'w')
f.write(custom_action)
f.close()

#%store custom_action > actions.py

### 6. Visualize the graph of all possible intents and responses -- using domain.yml and stories.md

In [15]:
!rasa visualize

c:\users\navan\anaconda3\lib\site-packages\rasa\utils\train_utils.py:628: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss. It will be set to `True` by default, Rasa Open Source 3.0.0 onwards.
  rasa.shared.utils.io.raise_warning(
c:\users\navan\anaconda3\lib\site-packages\rasa\utils\train_utils.py:601: UserWarning: model_confidence is set to `softmax`. It is recommended to try using `model_confidence=linear_norm` to make it easier to tune fallback thresholds.
  rasa.shared.utils.io.raise_warning(
c:\users\navan\anaconda3\lib\site-packages\rasa\core\policies\form_policy.py:51: FutureWarning: 'FormPolicy' is deprecated and will be removed in in the future. It is recommended to use the 'RulePolicy' instead. (will be removed in 3.0.0)
  rasa.shared.utils.io.raise_deprecation_warning(
c:\users\navan\anaconda3\lib\site-packages\rasa\shared\utils\io.py:97: UserWarning: It is not recommended to use the 'RulePolicy' with